# Setup

In [ ]:
!git clone https://github.com/mglnsk/fasterrcnn-pytorch-training-pipeline

fatal: destination path 'fasterrcnn-pytorch-training-pipeline' already exists and is not an empty directory.


In [3]:
%cd fasterrcnn-pytorch-training-pipeline

/content/fasterrcnn-pytorch-training-pipeline


In [3]:
!curl -L -o neu-steel.zip https://www.kaggle.com/api/v1/datasets/download/sovitrath/neu-steel-surface-defect-detect-trainvalid-split; unzip neu-steel.zip -d ./data/steel && rm neu-steel.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 26.3M  100 26.3M    0     0  27.6M      0 --:--:-- --:--:-- --:--:-- 27.6M
Archive:  neu-steel.zip
  inflating: ./data/steel/train_annotations/crazing_10.xml  
  inflating: ./data/steel/train_annotations/crazing_100.xml  
  inflating: ./data/steel/train_annotations/crazing_101.xml  
  inflating: ./data/steel/train_annotations/crazing_102.xml  
  inflating: ./data/steel/train_annotations/crazing_103.xml  
  inflating: ./data/steel/train_annotations/crazing_104.xml  
  inflating: ./data/steel/train_annotations/crazing_105.xml  
  inflating: ./data/steel/train_annotations/crazing_106.xml  
  inflating: ./data/steel/train_annotations/crazing_107.xml  
  inflating: ./data/steel/train_annotations/crazing_108.xml  
  inflating: ./data/steel/train_annotatio

In [4]:
%pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.7/458.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 132.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

# data config

In [4]:
%%writefile data_configs/neu_defect.yml
TRAIN_DIR_IMAGES: 'data/steel/train_images'
TRAIN_DIR_LABELS: 'data/steel/train_annotations'
VALID_DIR_IMAGES: 'data/steel/valid_images'
VALID_DIR_LABELS: 'data/steel/valid_annotations'

# Class names.
CLASSES: [
    '__background__',
    'crazing',
    'inclusion',
    'patches',
    'pitted_surface',
    'rolled-in_scale',
    'scratches'
]

# Number of classes (object classes + 1 for background class in Faster RCNN).
NC: 7

# Whether to save the predictions of the validation set while training.
SAVE_VALID_PREDICTION_IMAGES: True

Writing data_configs/neu_defect.yml


In [5]:
# verify yml config
import yaml
import os
config_data = {}
with open('data_configs/neu_defect.yml', 'r') as file_descriptor:
    config_data = yaml.safe_load(file_descriptor)

labels = ['TRAIN_DIR_LABELS', 'VALID_DIR_LABELS']
images = ['TRAIN_DIR_IMAGES', 'VALID_DIR_IMAGES']
dirs = labels + images
missing = False
for dir in dirs:
  p = config_data[dir]
  if os.path.exists(p):
    print(f"{dir} OK")
  else:
    print(f"missing path {p}")
    missing = True

assert (not missing), "ERROR invalid data config | missing directories"

print("OK all paths exists")

label_files_exists = False
image_files_exists = False
for dir in labels:
  for fil in os.listdir(config_data[dir]):
    if "xml" in fil:
      label_files_exists = True
      break
for dir in images:
  for fil in os.listdir(config_data[dir]):
    if "jpg" in fil:
      image_files_exists = True
      break
assert label_files_exists, "No label files in label directory"
assert image_files_exists, "No image files in image directory"

TRAIN_DIR_LABELS OK
VALID_DIR_LABELS OK
TRAIN_DIR_IMAGES OK
VALID_DIR_IMAGES OK
OK all paths exists


In [ ]:
!python train.py --epochs 25 \
                 --batch 24 \
                 --data data_configs/neu_defect.yml \
                 --model fasterrcnn_mobilenetv3_large_fpn \
                 --name resnet_defect_batch_mosaic \
                 --label-type pascal_voc \
                 -dw
                 #--mosaic 2 \

2025-07-20 06:33:00.885661: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752993180.905882    6545 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752993180.912132    6545 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-20 06:33:00.933163: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Not using distributed mode
device cuda
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331